In [1]:
import cv2
import sys
from keras.models import load_model
import skimage

Using TensorFlow backend.


In [2]:
cascPath = "./haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascPath)
from keras.models import model_from_json
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
json_file = open('fer.json','r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
#load woeights into new model
loaded_model.load_weights("fer.h5")
print("Loaded Model from disk")

#compile and evaluate loaded model
loaded_model.compile(loss=categorical_crossentropy,optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),metrics=['accuracy'])

Loaded Model from disk


In [3]:
from skimage import transform

In [4]:
def crop(image, faces, k=0):
    faces_arrays = []
    for (top, right, bottom, left)in faces:
        x0, y0 = left, bottom
        x1, y1 = right, top
        w, h = right-left, top-bottom
        cv2.rectangle(img=image, pt1=(x0, y0), pt2=(x1, y1), color=(255,0,0), thickness=2)
    # crop the region of interest over a copy 
        face = image[x1-20:x1+x0+20,y1-20:y1+y0+20].copy()
        faces_arrays.append(face)
    return faces_arrays

In [5]:
from PIL import Image,ImageDraw
import numpy as np

In [36]:
cap = cv2.VideoCapture(0)
zz=-1
while(True):
    zz+=1
    # Capture frame-by-frame
    ret, frame = cap.read()
    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.2,
        minNeighbors=4,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
    )
    f=faces
    img=gray
    fa=crop(img,f)
    ans=[]
    for i in fa:
        if i is not None:
            a=Image.fromarray(i)
            q=a.resize((48,48))
            #q.show()
            x=np.asarray(q)
            #print(x)
            x.setflags(write=1)
            x=x.astype('float64')
            #print(x.shape)
            x -= np.mean(x, axis=0)
            x /= (np.std(x, axis=0)+1e-100)
            x=x.reshape((1,48,48,1))
            #print(x)
            y=loaded_model.predict(x)
            z=0
            z1=-1
            y[0][6]-=0.7
            for j in range(0,7):
                if z<y[0][j]:
                    z1=j
                    z=y[0][j]
            ans.append(z1)
    # Draw a rectangle around the faces
    i=0
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x-30, y-30), (x+w+30, y+h+30), (0,250,90), 2)
        if ans[i]==0:
            cv2.putText(frame, "ANGRY", (x,y+h+60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255))
        if ans[i]==1:
            cv2.putText(frame, "DISGUST", (x,y+h+60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255))
        if ans[i]==2:
            cv2.putText(frame, "FEAR", (x,y+h+60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255))
        if ans[i]==3:
            cv2.putText(frame, "HAPPY", (x,y+h+60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255))
        if ans[i]==4:
            cv2.putText(frame, "SAD", (x,y+h+60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255))
        if ans[i]==5:
            cv2.putText(frame, "SURPRISE", (x,y+h+60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255))
        if ans[i]==6:
            cv2.putText(frame, "NEUTRAL", (x,y+h+60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255))
        i+=1
    # Display the resulting frame
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()